In [0]:
"""A very simple MNIST classifier.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/beginners
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None


In [3]:
# Import data
data_dir = '/tmp/tensorflow/mnist/input_data'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz

In [0]:
tf.set_random_seed(1)
seed = 1

两层神经网络

In [27]:
# Create the model
x = tf.placeholder(tf.float32, [None,784])
W1 = tf.Variable(tf.truncated_normal([784, 256],stddev=np.sqrt(2.0 / 784),seed =1)) #对每层的每个神经元神经元初始化，落在均值附近
b1 = tf.Variable(tf.zeros([256]))
Z1 = tf.add(tf.matmul(x,W1),b1) 
A1 = tf.nn.relu(Z1) 
W2 = tf.Variable(tf.truncated_normal([256, 10],stddev=np.sqrt(2.0 / 256),seed = 1))
b2 = tf.Variable(tf.zeros([10]))
Z2 = tf.add(tf.matmul(A1,W2),b2)

y_ = tf.placeholder(tf.float32, [None,10])

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=Z2))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

# Train
for _ in range(3000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
  
   # Test trained model
correct_prediction = tf.equal(tf.argmax(Z2, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y_: mnist.test.labels}))

0.98


将这个模型优化至98%以上的准确率。
Hint：
- 多隐层
- 激活函数
- 正则化
- 初始化
- 摸索一下各个超参数
  - 隐层神经元数量
  - 学习率
  - 正则化惩罚因子
  - 最好每隔几个step就对loss、accuracy等等进行一次输出，这样才能有根据地进行调整

两层神经网络+正则

In [37]:
# Create the model
x = tf.placeholder(tf.float32, [None,784])
W1 = tf.Variable(tf.truncated_normal([784, 256],stddev=np.sqrt(2.0 / 784),seed =1)) 
b1 = tf.Variable(tf.zeros([256]))
Z1 = tf.add(tf.matmul(x,W1),b1) 
A1 = tf.nn.relu(Z1) 
W2 = tf.Variable(tf.truncated_normal([256, 10],stddev=np.sqrt(2.0 / 256),seed = 1))
b2 = tf.Variable(tf.zeros([10]))
Z2 = tf.add(tf.matmul(A1,W2),b2)


y_ = tf.placeholder(tf.float32, [None,10])

beta = 0.0001
reg = tf.nn.l2_loss(W1)+tf.nn.l2_loss(W2)

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=Z2)+ beta*reg)

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

# Train
for _ in range(3000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
  
   # Test trained model
correct_prediction = tf.equal(tf.argmax(Z2, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y_: mnist.test.labels}))

0.9798


四层神经网络(由于数据量太小，随着神经网络层数的增加，需要的训练数据越多，对于这个数据集一层隐层是最合适的)

In [54]:
# Create the model
x = tf.placeholder(tf.float32, [None,784])

W1 = tf.Variable(tf.truncated_normal([784, 256],stddev=np.sqrt(2.0 / 256),seed =1))
b1 = tf.Variable(tf.zeros([256]))
Z1 = tf.add(tf.matmul(x,W1),b1) 
A1 = tf.nn.relu(Z1) 

W2 = tf.Variable(tf.truncated_normal([256, 128],stddev=np.sqrt(2.0 / 128),seed =1))
b2 = tf.Variable(tf.zeros([128]))
Z2 = tf.add(tf.matmul(A1,W2),b2)
A2 = tf.nn.relu(Z2)

W3 = tf.Variable(tf.truncated_normal([128, 64],stddev=np.sqrt(2.0 / 64),seed =1))
b3 = tf.Variable(tf.zeros([64]))
Z3 = tf.add(tf.matmul(A2,W3),b3)
A3 = tf.nn.relu(Z3)

W4 = tf.Variable(tf.truncated_normal([64, 10],stddev=np.sqrt(2.0 / 10),seed =1))
b4 = tf.Variable(tf.zeros([10]))
Z4 = tf.add(tf.matmul(A3,W4),b4)

beta = 0.001
reg = tf.nn.l2_loss(W1)+tf.nn.l2_loss(W2)+tf.nn.l2_loss(W3)+tf.nn.l2_loss(W4)

y_ = tf.placeholder(tf.float32, [None,10])
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=Z4)+ beta*reg)

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

# Train
for _ in range(3000):
  batch_xs, batch_ys = mnist.train.next_batch(100) 
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
  
    # Test trained model
correct_prediction = tf.equal(tf.argmax(Z4, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y_: mnist.test.labels}))

0.098


调整学习率（手动调节）

In [57]:
# Create the model
x = tf.placeholder(tf.float32, [None,784])
W1 = tf.Variable(tf.truncated_normal([784, 256],stddev=np.sqrt(2.0 / 784),seed =1)) #对每层的每个神经元神经元初始化，落在均值附近
b1 = tf.Variable(tf.zeros([256]))
Z1 = tf.add(tf.matmul(x,W1),b1) 
A1 = tf.nn.relu(Z1) 
W2 = tf.Variable(tf.truncated_normal([256, 10],stddev=np.sqrt(2.0 / 256),seed = 1))
b2 = tf.Variable(tf.zeros([10]))
Z2 = tf.add(tf.matmul(A1,W2),b2)

y_ = tf.placeholder(tf.float32, [None,10])

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=Z2))

train_step = tf.train.GradientDescentOptimizer(0.2).minimize(cross_entropy)

sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

# Train
for _ in range(3000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
  
   # Test trained model
correct_prediction = tf.equal(tf.argmax(Z2, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                      y_: mnist.test.labels}))

0.9736
